<h1> Training the Electric Birder on the God damned cloud! </h1>

Pardon my French. This has been a very long time coming :/

In [ ]:
!pip install keras

In [ ]:
!pip install tensorflow --upgrade

In [ ]:
# !gcloud components update

In [1]:
BUCKET = 'electric-birder-71281-bird-db'
PROJECT = 'electric-birder-71281'
REGION = 'us-east1'

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '1.13'  

In [3]:
%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


In [4]:
%bash
gsutil ls "gs://electric-birder-71281-bird-db/Birds_dB/Mappings/"

gs://electric-birder-71281-bird-db/Birds_dB/Mappings/allfiles.csv
gs://electric-birder-71281-bird-db/Birds_dB/Mappings/bird_class_list.txt
gs://electric-birder-71281-bird-db/Birds_dB/Mappings/eval_set_cloud.csv
gs://electric-birder-71281-bird-db/Birds_dB/Mappings/minimal_bird_list.txt
gs://electric-birder-71281-bird-db/Birds_dB/Mappings/test_set_cloud.csv
gs://electric-birder-71281-bird-db/Birds_dB/Mappings/train_set_cloud.csv


In [ ]:
%bash
echo "bucket=${BUCKET}"
#rm -rf babyweight_trained
export PYTHONPATH=${PYTHONPATH}:${PWD}
python -m train_model.task \
 # --bucket=${BUCKET} \
  #--output_dir=babyweight_trained \
  #--job-dir=./tmp \
  #--pattern="00000-of-" --train_examples=1 --eval_steps=1

In [7]:
!gsutil defacl set public-read gs://$BUCKET
!gsutil defacl set public-read gs://electric-birder-71281


Setting default object ACL on gs://electric-birder-71281-bird-db/...
Setting default object ACL on gs://electric-birder-71281/...                    


In [8]:
%%bash
echo "bucket=${BUCKET}"
export PYTHONPATH=${PYTHONPATH}:${PWD}


bucket=electric-birder-71281-bird-db


In [27]:
from google.datalab.ml import TensorBoard
TensorBoard().start("gs://electric-birder-71281/Estimator_Output/4")
                    

Exception: Cannot start TensorBoard.

In [ ]:
!gcloud ml-engine local train \
    --module-name=train_model.task \
    --package-path=${PWD}/train_model \
    -- \
    --batch_size=5 \

In [23]:
for pid in TensorBoard.list()["pid"]:
    TensorBoard().stop(pid)
    print("Stopped TensorBoard with pid {}".format(pid))

KeyError: 'pid'

# Training on the ML Engine

In [11]:
%%bash
OUTDIR=gs://electric-birder-71281/Estimator_Output/4/
JOBNAME=KKJ_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
# gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
    --region=$REGION \
    --module-name=train_model.task \
    --package-path=${PWD}/train_model \
    --job-dir=$OUTDIR \
    --staging-bucket=gs://electric-birder-71281 \
    --scale-tier=BASIC_GPU \
    --runtime-version=$TFVERSION \
    -- \
    --output_dir=$OUTDIR \
    --batch_size=40 \


gs://electric-birder-71281/Estimator_Output/4/ us-east1 KKJ_190427_124056
jobId: KKJ_190427_124056
state: QUEUED


Job [KKJ_190427_124056] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe KKJ_190427_124056

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs KKJ_190427_124056
